In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
train=pd.read_csv("drive/My Drive/colabdrive/GUE/GUE/prom/prom_core_tata/train.csv")
train.head()

,sequence,label
0,ACTACAAATGGACGAGAGAGGCGGCCGTCCATTAGTTAGCGGCTCC...,1
1,AAAATATAGGCCGGGGTACCTCAGCCGGAAGGGACTTCAGTTAGTG...,1
2,CCTACATAAGTCCCTGTATAAAGTCACTGACCCATTTGCACTGCTG...,1
3,AGTTTAAAAGCCAGCCAGTCATACTAAAAAAAAGAATTCAGGTTTT...,0
4,GGGATAAGAAGGACAGAGAGAGACTGTAGGAAGTCAAGGGGTGGAG...,0


In [ ]:
test=pd.read_csv("drive/My Drive/colabdrive/GUE/GUE/prom/prom_core_tata/test.csv")
test.head()

,sequence,label
0,GCAATAAAAGGCTTAGCCACATAGTGCATGCAGAGCTCTGACTTCT...,0
1,ATTATATATGTCTATATTTTTTGCAGCCATTATGCAGATGTAGATC...,0
2,TCTTTAAATACAAGTGAGTTTGGAGAAAAGAAAAAGGGGGAGAAGG...,0
3,AGGTTAAATAGGGGTTGAGATATGATGCTCAGGAGAAGCGCTTTCT...,1
4,GGGAGAAAAACGGGGAACCAGGAATGTTGGGAGGGGGACGAGGAAT...,0


In [ ]:
dev=pd.read_csv("drive/My Drive/colabdrive/GUE/GUE/prom/prom_core_tata/dev.csv")
dev.head()

,sequence,label
0,TGCTTAAATACCCGACCTTGCTGCACCAGAGAAAGAAGCAGCGATT...,0
1,TATAAAACAGCTCAATCGGGGAGTACAACCTTCGGTTTCTCTTCGG...,1
2,CCCTTTAAAAAGACGGGAAAGGAGGAGTCCAGTGCTGGAGGGGCAG...,1
3,AATTATAATTTCCTGGGAAAGTGTCTGGTTCTCGGAGAATCCTTGG...,1
4,CCTTTATAAAGCACAGACTTTCTATTTCACTCCGCGGTATCTGCAT...,1


In [ ]:
!pip install transformers

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M",num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.8.attention.self.value.weight', 'bert.encoder.layer.2.attention.self.key.weight', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.4.output.LayerNorm.bias', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.9.output.LayerNorm.bias', 'bert.encoder.layer.4.output.dense.bias', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.3.output.dense.bias', 'bert.encoder.layer.4.output.dense.weight', 'bert.encoder.layer.10.output.LayerNorm.weight', 'bert.encoder.layer.9.output.dense.weight', 'bert.encoder.layer.3.attention.self.key.bias', 'b

In [ ]:
!pip install torch
import torch
from torch.utils.data import Dataset, DataLoader,random_split

In [ ]:
class DNADataset(Dataset):
    def __init__(self, sequence, label, tokenizer, max_length):
        self.sequence = sequence
        self.label = label
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, idx):
        sequence = self.sequence[idx]
        label = self.label[idx]
        encoding = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'sequence': sequence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_length, batch_size):
    ds = DNADataset(
        sequence=df.sequence.to_numpy(),
        label=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_length=max_length
    )
    return DataLoader(
        ds,
        batch_size=batch_size
    )

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

In [ ]:
MAX_LEN = 128
dataset = DNADataset(train.sequence, train.label, tokenizer, MAX_LEN)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# Define the training arguments
training_args = TrainingArguments(
    output_dir='/drive/MyDrive/colabdrive/results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='/drive/MyDrive/colabdrive/logs',
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,0.445800


TrainOutput(global_step=615, training_loss=0.41637371652494604, metrics={'train_runtime': 391.4151, 'train_samples_per_second': 50.113, 'train_steps_per_second': 1.571, 'total_flos': 1290242422183680.0, 'train_loss': 0.41637371652494604, 'epoch': 5.0})

In [ ]:
# Evaluation
eval_results=trainer.evaluate()

In [ ]:
# Print the accuracy
print(f'Evaluation Accuracy: {eval_results["eval_accuracy"]}')

Evaluation Accuracy: 0.7849133537206932


In [ ]:
# Save the model
model.save_pretrained('/drive/MyDrive/colabdrive/models')


In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('/drive/MyDrive/colabdrive/models')

In [ ]:
# Make predictions on new data
sequence = "CACTTTATAAACATCCCCAGCCTGTGAGAGCAGAGGGCAGGGAGATAGTGTGAGACAGGAGCCCAGGGGA"
encoding = tokenizer.encode_plus(
    sequence,
    add_special_tokens=True,
    max_length=MAX_LEN,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
logits = outputs.logits
predicted_label = torch.argmax(logits, dim=1)

print(f'The predicted label for the sequence is: {predicted_label.item()}')


The predicted label for the sequence is: 1
